<a href="https://colab.research.google.com/github/NateMophi/SCC-454/blob/main/LAB2/SCC454_Lab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install pyspark==4.0.0 -q

# Java Installation
!apt-get install openjdk-11-jdk-headless -qq > /dev/null

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.1/434.1 MB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
E: Failed to fetch http://security.ubuntu.com/ubuntu/pool/main/o/openjdk-lts/openjdk-11-jre-headless_11.0.29%2b7-1ubuntu1%7e22.04_amd64.deb  404  Not Found [IP: 91.189.92.22 80]
E: Failed to fetch http://security.ubuntu.com/ubuntu/pool/main/o/openjdk-lts/openjdk-11-jdk-headless_11.0.29%2b7-1ubuntu1%7e22.04_amd64.deb  404  Not Found [IP: 91.189.92.22 80]
E: Unable to fetch some archives, maybe run apt-get update or try with --fix-missing?


In [4]:
# Set Java environmenr variable
import os
os.environ["JAVA HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"

print("PySpark & Java installed successfully!")

PySpark & Java installed successfully!


In [5]:
from pyspark.sql import SparkSession
from pyspark import SparkConf

# Session Creation
spark = SparkSession.builder.appName("SCC454-SparkIntro").config("spark.driver.memory", "4g")\
        .config("spark.ui.port", "4050")\
        .getOrCreate()

  # Underlying Context
sc = spark.sparkContext
print(f"Spark Version: {spark.version}")
print(f"Spark App Name: {spark.sparkContext.appName}")
print(f"Master: {spark.sparkContext.master}")

Spark Version: 4.0.0
Spark App Name: SCC454-SparkIntro
Master: local[*]


# **Resilient Distributed Datasets (RDDs)**

In [6]:
# RDD Creation

# 1a) Parallelization from a Python Collection
nums = [1,2,3,4,5,6,7,8,9,10]
nums_rdd =sc.parallelize(nums)

type(nums_rdd)
print(f"Number of paritions: {nums_rdd.getNumPartitions()}")
print(f"First 5 elements : {nums_rdd.take(5)}")



Number of paritions: 2
First 5 elements : [1, 2, 3, 4, 5]


In [7]:
# 1b) Parallelize with a specific number of partitions
nums_rdd_4part = sc.parallelize(nums, 4)
print(f"Number of partitions: {nums_rdd_4part.getNumPartitions()}")

# View Partitions
print("\nData in each partition: ")
print(nums_rdd_4part.glom().collect())

Number of partitions: 4

Data in each partition: 
[[1, 2], [3, 4], [5, 6], [7, 8, 9, 10]]


In [8]:
# 2) Create RDD from text file
sample_text = """Apache Spark is a unified analytics engine for large-scale data processing.
It provides high-level APIs in Java, Scala, Python and R.
Spark powers a stack of libraries including SQL and DataFrames.
It also includes MLlib for machine learning and GraphX for graph processing.
Spark can run on Hadoop, Apache Mesos, Kubernetes, standalone, or in the cloud."""


with open("spark_intro.txt", "w") as f:
  f.write(sample_text)

# Load text file as RDD
text_rdd = sc.textFile("spark_intro.txt")
print(f"Number of lines: {text_rdd.count()}")
print("\n1st 3 lines:")
for line in text_rdd.take(3):
  print(f" - {line}")

Number of lines: 5

1st 3 lines:
 - Apache Spark is a unified analytics engine for large-scale data processing.
 - It provides high-level APIs in Java, Scala, Python and R.
 - Spark powers a stack of libraries including SQL and DataFrames.


## 2.3 Transformations and Actions

RDD operations are divided into two categories:

**Transformations**: Create a new RDD from an existing one. They are *lazy* - they don't execute until an action is called.

| Transformation | Description |
|----------------|-------------|
| `map(func)` | Apply function to each element |
| `filter(func)` | Keep elements where function returns true |
| `flatMap(func)` | Map then flatten the results |
| `distinct()` | Remove duplicates |
| `reduceByKey(func)` | Aggregate values by key |
| `groupByKey()` | Group values by key |
| `sortBy(func)` | Sort RDD elements |

**Actions**: Return a value to the driver program. They *trigger* the execution of transformations.

| Action | Description |
|--------|-------------|
| `collect()` | Return all elements as a list |
| `count()` | Return the number of elements |
| `first()` | Return the first element |
| `take(n)` | Return first n elements |
| `reduce(func)` | Aggregate elements using a function |
| `saveAsTextFile(path)` | Write elements to a text file |

In [9]:
numbers = sc.parallelize(range(1,11))

# map: Apply function to each element
squared = numbers.map(lambda x: x**2)
print(f"Original: {numbers.collect()} ")
print(f"Squared: {squared.collect()} ")

# filter: Keep Only elements that match a condition
evens = numbers.filter(lambda x: x%2==0)
print(f"Evens: {evens.collect()} ")

# Chaining transformation
even_squares = numbers.filter(lambda x: x%2==0).map(lambda x: x**2)
print(f"Even squares: {even_squares}")




Original: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10] 
Squared: [1, 4, 9, 16, 25, 36, 49, 64, 81, 100] 
Evens: [2, 4, 6, 8, 10] 


In [11]:
# flatMap: map then Flatten
sentences = sc.parallelize(["Hello World", "Apache Spark", "Big Data"])

# map vs flatmap
words_map = sentences.map(lambda s: s.split())
words_flatmap = sentences.flatMap(lambda s: s.split())

print(f"Using map:     {words_map.collect()}")
print(f"Using flatMap: {words_flatmap.collect()}")

Using map:     [['Hello', 'World'], ['Apache', 'Spark'], ['Big', 'Data']]
Using flatMap: ['Hello', 'World', 'Apache', 'Spark', 'Big', 'Data']


In [12]:
sentences

ParallelCollectionRDD[13] at readRDDFromFile at PythonRDD.scala:297

In [13]:
# Action Examples

numbers = sc.parallelize([1, 2, 3, 4, 5])

print(f"collect(): {numbers.collect()}")
print(f"count():   {numbers.count()}")
print(f"first():   {numbers.first()}")
print(f"take(3):   {numbers.take(3)}")
print(f"sum():     {numbers.sum()}")
print(f"mean():    {numbers.mean()}")
print(f"max():     {numbers.max()}")
print(f"min():     {numbers.min()}")

# reduce: Aggregate elements
total = numbers.reduce(lambda a, b: a + b)
print(f"reduce(+): {total}")

collect(): [1, 2, 3, 4, 5]
count():   5
first():   1
take(3):   [1, 2, 3]
sum():     15
mean():    3.0
max():     5
min():     1
reduce(+): 15


## 2.4 Lazy Evaluation

Spark transformations are *lazy* - they don't execute immediately. Instead, Spark builds up a *lineage graph* (DAG - Directed Acyclic Graph) of transformations. The actual computation only happens when an action is called.

**Benefits of Lazy Evaluation:**
- Allows Spark to optimize the execution plan
- Reduces unnecessary computations
- Enables fault tolerance through lineage

In [20]:
# Demonstrating Lazy Evaluation
import time

# Create an RDD and apply transformations
print("Creating RDD and transformations...")
start = time.time()

large_rdd = sc.parallelize(range(1000000))
transformed = large_rdd.map(lambda x: x * 2).filter(lambda x: x % 4 == 0)

print(f"Time to define transformations: {time.time() - start:.4f} seconds")
print(f"Transformations are defined but NOT executed yet!")
print(f"Type of 'transformed': {type(transformed)}")

# Now trigger execution with an action
print("\nCalling count() action...")
start = time.time()
result = transformed.count()
print(f"Time to execute: {time.time() - start:.4f} seconds")
print(f"Count: {result}")

Creating RDD and transformations...
Time to define transformations: 0.0061 seconds
Transformations are defined but NOT executed yet!
Type of 'transformed': <class 'pyspark.core.rdd.PipelinedRDD'>

Calling count() action...
Time to execute: 1.1921 seconds
Count: 500000


## 2.5 Word Count Example - The "Hello World" of Big Data

Word Count is the classic MapReduce example. Let's implement it using Spark RDDs.

In [21]:
# Create a sample text
text = """Spark is fast and general purpose cluster computing system
Spark provides high level APIs in Java Scala Python and R
Spark supports general computation graphs for data analysis
Spark has rich set of higher level tools including Spark SQL
Spark SQL provides support for structured data processing"""

# Save to file
with open("wordcount_input.txt", "w") as f:
    f.write(text)

In [28]:
# Word Count using RDDs - Step by Step
lines = sc.textFile("wordcount_input.txt")

# Step 1: Split each line into words
words = lines.flatMap(lambda line: line.lower().split())
print("Step 1 - Words:")
print(words.take(10))

# Step 2: Map each word to a (word, 1) pair
word_pairs = words.map(lambda word: (word, 1))
print("\nStep 2 - Word pairs:")
print(word_pairs.take(10))

# Step 3: Reduce by key - sum up counts for each word
word_counts = word_pairs.reduceByKey(lambda a, b: a + b)
print("\nStep 3 - Word counts:")
print(word_counts.take(10))

# Step 4: Sort by count (descending)
sorted_counts = word_counts.sortBy(lambda x: -x[1])
print("\nTop 10 words:")
for word, count in sorted_counts.take(10):
    print(f"  {word}: {count}")

Step 1 - Words:
['spark', 'is', 'fast', 'and', 'general', 'purpose', 'cluster', 'computing', 'system', 'spark']

Step 2 - Word pairs:
[('spark', 1), ('is', 1), ('fast', 1), ('and', 1), ('general', 1), ('purpose', 1), ('cluster', 1), ('computing', 1), ('system', 1), ('spark', 1)]

Step 3 - Word counts:
[('fast', 1), ('and', 2), ('general', 2), ('computing', 1), ('high', 1), ('level', 2), ('java', 1), ('python', 1), ('supports', 1), ('for', 2)]

Top 10 words:
  spark: 6
  and: 2
  general: 2
  level: 2
  for: 2
  provides: 2
  data: 2
  sql: 2
  fast: 1
  computing: 1


In [29]:
# Word Count - Concise Version (one-liner)

word_counts_concise = sc.textFile("wordcount_input.txt") \
    .flatMap(lambda line: line.lower().split()) \
    .map(lambda word: (word, 1)) \
    .reduceByKey(lambda a, b: a + b) \
    .sortBy(lambda x: -x[1])

print("Word counts (concise version):")
for word, count in word_counts_concise.take(10):
    print(f"  {word}: {count}")

Word counts (concise version):
  spark: 6
  and: 2
  general: 2
  level: 2
  for: 2
  provides: 2
  data: 2
  sql: 2
  fast: 1
  computing: 1


---
# Part 3: Spark DataFrames
---

## 3.1 Introduction to DataFrames

A DataFrame is a distributed collection of data organized into named columns, similar to a table in a relational database or a DataFrame in pandas. DataFrames are built on top of RDDs but provide:

- **Schema**: Named columns with data types
- **Optimized Execution**: Catalyst optimizer for query optimization
- **Familiar API**: SQL-like operations

**DataFrames vs RDDs:**

| Feature | RDD | DataFrame |
|---------|-----|----------|
| Schema | No schema | Named columns with types |
| Optimization | No automatic optimization | Catalyst optimizer |
| Ease of use | Low-level API | High-level, SQL-like API |
| Performance | Good | Better (optimized) |
| Data types | Any Python object | Structured data types |

## 3.2 Creating DataFrames

There are multiple ways to create DataFrames in Spark.

In [30]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType
from pyspark.sql import Row

# Method 1: From a list of tuples
data = [
    ("Alice", 28, "Data Scientist", 75000),
    ("Bob", 35, "Software Engineer", 85000),
    ("Charlie", 32, "Data Analyst", 65000),
    ("Diana", 29, "ML Engineer", 90000),
    ("Eve", 41, "Data Scientist", 95000)
]
columns = ["name", "age", "role", "salary"]

df = spark.createDataFrame(data, columns)
df.show()


+-------+---+-----------------+------+
|   name|age|             role|salary|
+-------+---+-----------------+------+
|  Alice| 28|   Data Scientist| 75000|
|    Bob| 35|Software Engineer| 85000|
|Charlie| 32|     Data Analyst| 65000|
|  Diana| 29|      ML Engineer| 90000|
|    Eve| 41|   Data Scientist| 95000|
+-------+---+-----------------+------+



In [31]:
# Method 2: With explicit schema
schema = StructType([
    StructField("name", StringType(), True),
    StructField("age", IntegerType(), True),
    StructField("role", StringType(), True),
    StructField("salary", IntegerType(), True)
])

df_with_schema = spark.createDataFrame(data, schema)
df_with_schema.printSchema()

root
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- role: string (nullable = true)
 |-- salary: integer (nullable = true)

